# Rest API - Advanced Hunting Query

<table align="left">
  <tr>
    <th>Type</th>
      <th>Title</th>
    <th>Description</th>
  </tr>
  <tr>
      <td>Documentation</td>
    <td><b><a href="https://docs.microsoft.com/en-us/windows/security/threat-protection/microsoft-defender-atp/run-advanced-query-api">Advanced Hunting API</a></td>
    <td>This is the documentation that refers to what permissions is required and request examples you can use to query the MDATP API </td>
  </tr>
</table>

## Pre-requisite

In [ ]:
$requiredModules = ('Microsoft.graph')

Install-Module $requiredModules

In [ ]:
Select-MgProfile -Name "beta"

In [ ]:
Get-MGGroup

In [ ]:
Get-MGContext

In [ ]:
(Get-MGContext).scopes

In [ ]:
# Connect to the MS Graph API using device code authentication
Connect-MGGraph -Scopes Group.Read.All -ContextScope Process -ForceRefresh

In [ ]:
$scope = "DeviceManagementManagedDevices.ReadWrite.All"

Connect-MGGraph -Scopes $scope -ContextScope Process -ForceRefresh

In [ ]:
## Find Command
Get-Command -Module Microsoft.Graph* *command*

In [ ]:
Get-MgDeviceCommand

## Import required modules and credentials 

In [ ]:
$requiredModules = @('Microsoft.Graph')

## This will run a script to import the required modules and credentials stored in the "credential" file
$scriptPath = "~/Notebooks/PowerShell/custom-modules/import.ps1"
. $scriptPath  # this will run the script

## Authenticate with Graph API and get access Token for MDATP Resource

In [ ]:
## Get a token if authenticates properly. 
$authParams = @{
    tenantId = $credentials.tenantId
    client_id = $credentials.client_id
    client_secret = $credentials.client_secret
    resource = "https://api.securitycenter.windows.com"    ## resource Dont change since we want to query MDATP REST API Resource
    grant_type = "client_credentials"  ## This is using a appliation ID and secret to authenticate
};

## this will authenticate with Graph API and token will be stored in <tokenResponse> variable.
$tokenResponse = Invoke-GraphAuthentication -authParams $authParams

### Put your Hunting Query here

In [ ]:
## Enter your query here below $Query
$body = (@{'Query' = @"

DeviceNetworkEvents                                          
| where InitiatingProcessFileName =~ 'powershell.exe'
| limit 10

"@}) | ConvertTO-JSON;

## Query the MDATP API

### Query the MDATP API Grab report schema 

In [ ]:
$URI = "https://api.securitycenter.windows.com/api/advancedqueries/run"   ## This is the URL to query with the access token.
$header = @{                                                              ## This is the header
    'Content-Type' = 'application/json'
    Accept = 'application/json'
    Authorization = "Bearer $($tokenResponse.access_token)" }

$Response = (Invoke-RestMethod -Method POST -Uri $URI -Headers $header -body $body -ErrorAction Stop)
$Response

### Parse Results

In [ ]:
## Filter by these columns
$includeColumns = @('Timestamp','DeviceName','ActionType','RemoteIP')

## These are the results
$Result.Results | Select-Object -Property $includeColumns